In [1]:
import numpy as np
import pandaqws as pd
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.preprocessing import PolynomialFeatures
from matplotlib import pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score,GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,f1_score
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib

In [2]:
df = pd.read_csv('current_FAO/raw_files/Production_Crops_E_All_Data_(Normalized).csv', encoding = 'ISO-8859-1')
df.head()

,Area Code,Area,Item Code,Item,Element Code,Element,Year Code,Year,Unit,Value,Flag
0,2,Afghanistan,221,"Almonds, with shell",5312,Area harvested,1975,1975,ha,0.0,F
1,2,Afghanistan,221,"Almonds, with shell",5312,Area harvested,1976,1976,ha,5900.0,F
2,2,Afghanistan,221,"Almonds, with shell",5312,Area harvested,1977,1977,ha,6000.0,F
3,2,Afghanistan,221,"Almonds, with shell",5312,Area harvested,1978,1978,ha,6000.0,F
4,2,Afghanistan,221,"Almonds, with shell",5312,Area harvested,1979,1979,ha,6000.0,F


In [3]:
df.Element.unique()

array(['Area harvested', 'Yield', 'Production', 'Seed'], dtype=object)

In [4]:
df.drop(['Flag','Year Code','Element Code'], 1, inplace = True)
df

,Area Code,Area,Item Code,Item,Element,Year,Unit,Value
0,2,Afghanistan,221,"Almonds, with shell",Area harvested,1975,ha,0.0
1,2,Afghanistan,221,"Almonds, with shell",Area harvested,1976,ha,5900.0
2,2,Afghanistan,221,"Almonds, with shell",Area harvested,1977,ha,6000.0
3,2,Afghanistan,221,"Almonds, with shell",Area harvested,1978,ha,6000.0
4,2,Afghanistan,221,"Almonds, with shell",Area harvested,1979,ha,6000.0
...,...,...,...,...,...,...,...,...
2597257,5817,Net Food Importing Developing Countries,1800,"Vegetables&Melons, Total",Seed,2010,tonnes,17232.0
2597258,5817,Net Food Importing Developing Countries,1800,"Vegetables&Melons, Total",Seed,2011,tonnes,17976.0
2597259,5817,Net Food Importing Developing Countries,1800,"Vegetables&Melons, Total",Seed,2012,tonnes,15730.0
2597260,5817,Net Food Importing Developing Countries,1800,"Vegetables&Melons, Total",Seed,2013,tonnes,15325.0


In [5]:
df_area = df.loc[(df['Element']=='Area harvested')]
df_area.drop([ 'Element', 'Unit'], 1, inplace = True)
df_area.rename(columns = {'Value':'Area harvested(ha)'}, inplace = True) 

df_yield = df.loc[(df['Element']=='Yield')]
df_yield.drop([ 'Element', 'Unit'], 1, inplace = True)
df_yield.rename(columns = {'Value':'Yield(hg/ha)'}, inplace = True) 

df_production = df.loc[(df['Element']=='Production')]
df_production.drop([ 'Element', 'Unit'], 1, inplace = True)
df_production.rename(columns = {'Value':'Production(tonnes)'}, inplace = True) 

df_seed = df.loc[(df['Element']=='Seed')]

df_seed.drop([ 'Element', 'Unit'], 1, inplace = True)
df_seed.rename(columns = {'Value':'Seed(tonnes)'}, inplace = True) 

c:\users\bbcc\appdata\local\programs\python\python37\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
c:\users\bbcc\appdata\local\programs\python\python37\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [6]:
print(df_area)
print(df_yield)
print(df_production)
print(df_seed)

         Area Code                                     Area  Item Code  \
0                2                              Afghanistan        221   
1                2                              Afghanistan        221   
2                2                              Afghanistan        221   
3                2                              Afghanistan        221   
4                2                              Afghanistan        221   
...            ...                                      ...        ...   
2597095       5817  Net Food Importing Developing Countries       1800   
2597096       5817  Net Food Importing Developing Countries       1800   
2597097       5817  Net Food Importing Developing Countries       1800   
2597098       5817  Net Food Importing Developing Countries       1800   
2597099       5817  Net Food Importing Developing Countries       1800   

                             Item  Year  Area harvested(ha)  
0             Almonds, with shell  1975          

In [7]:
df_train= pd.merge(df_area, df_yield, on=['Area','Item Code', 'Year','Area Code','Item'])
df_train = pd.merge(df_train, df_production, on=['Area','Item Code','Year','Area Code','Item'])
df_train

,Area Code,Area,Item Code,Item,Year,Area harvested(ha),Yield(hg/ha),Production(tonnes)
0,2,Afghanistan,221,"Almonds, with shell",1976,5900.0,16610.0,9800.0
1,2,Afghanistan,221,"Almonds, with shell",1977,6000.0,15000.0,9000.0
2,2,Afghanistan,221,"Almonds, with shell",1978,6000.0,20000.0,12000.0
3,2,Afghanistan,221,"Almonds, with shell",1979,6000.0,17500.0,10500.0
4,2,Afghanistan,221,"Almonds, with shell",1980,5800.0,17069.0,9900.0
...,...,...,...,...,...,...,...,...
710482,5817,Net Food Importing Developing Countries,1800,"Vegetables&Melons, Total",2010,6658623.0,118603.0,78973325.0
710483,5817,Net Food Importing Developing Countries,1800,"Vegetables&Melons, Total",2011,6694770.0,119873.0,80252302.0
710484,5817,Net Food Importing Developing Countries,1800,"Vegetables&Melons, Total",2012,6751835.0,120797.0,81560479.0
710485,5817,Net Food Importing Developing Countries,1800,"Vegetables&Melons, Total",2013,6847771.0,120205.0,82313348.0


In [ ]:
'''
df_corr=df.drop(columns=['Area','Item'])
cm=np.corrcoef(df_corr.values.T)
sns.set(font_scale=1.5)
plt.figure(figsize=(14,14))
hm=sns.heatmap(cm,cbar=True,annot=True,square=True,fmt='.2f')
plt.show()
'''
